# Batch reduction script for kast data

Performs the following steps:
* generates instruction file and log sheet from data files
* conducts first pass check of source centering for instruction file editing
* conducts extraction and template fitting

In [ ]:
# imports
import kastredux as kr
import os

import glob
import numpy
import os
import pandas
import pickle
import scipy
from astropy.io import fits
from scipy.interpolate import interp1d
import splat 
import numpy,scipy,copy
from astropy.io import fits
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

## first generate baseline instruction and log files

In [ ]:
# data folder
date = '260110'
base = '/Users/adam/data/kast/'
dfolder = '{}{}/data/'.format(base,date)

In [ ]:
# Generate instruction file and logs
kr.makeInstructions(dfolder,verbose=True,savelog=True)

## now double check instruction files and make sure they align with your logs

Things to check
* source names
* correct telluric star is assigned
* correct files are assigned (won't catch "test" files)

Edit the instruction files accordingly

# RED SIDE

## Run profileCheck to make sure extractions are in the right place

Take a look at the "diagnostic_profile_*.pdf" files in the reductions folder to make sure things are lined up as you want them. Adjust the instructions file as necessary, which may include:
* changing the line centers
* changing the background widths
* adding additional traces; duplicate the source instruction line and change name to \[name\]-2, \[name\]-3, etc.
* if you have particularly faint sources, you may want to use the trace of the telluric calibrator. To do this, add these commands to the source instruction line: "RECENTER=False	APPLY_TRACE=True"

Do this iteratively until you are happy with the baseline extraction parameters

In [ ]:
# RED SIDE profile check
input_file = dfolder+'../reduction/input.txt'
par = kr.profileCheck(input_file,verbose=True)

## now run extraction code

Again, you may need to run this iteratively to address issues with extractions or source/background regions. Things to look out for:

* wavelength calibration gives large errors (> 1 Ang, >30 km/s) - this may be a misalignment in the initial guesses of the wavelength centers. Check the "diagnostic_wavecal_initial_RED.pdf' file (or BLUE), and if there is n issue you can address this by adding a new instruction line, replaceing wave and pixel numbers with the specific lines (see [https://mthamilton.ucolick.org/techdocs/instruments/kast/kast_arcSpectra.html]):

WAVE_INITIAL	WAVE=6402.250,6929.43,7173.960,7948.175,8495.360,8780.622	PIXEL=621,1059,1210,1814,2239,2430

* poor flux calibration - inspect the flux 
* background regions overlapping with other sources: adjust the range assigned to "BACK"  

In [ ]:
# RED SIDE extraction
input_file = dfolder+'../reduction/input.txt'
redux = kr.reduce(instructions=input_file,reset=True,verbose=True)

In [ ]:
# CLASSIFY RED SIDE
kr.initializeStandards([40,75])
kr.initializeStandards([40,75],sd=True)
kr.initializeStandards([40,75],esd=True)
par = kr.readInstructions(input_file)
sources = list(par['SOURCE'].keys())
for src in sources:
    sfile = '{}/kast{}_{}_{}.fits'.format(par['REDUCTION_FOLDER'],par['MODE'],src,par['DATE'])
    if os.path.exists(sfile):
        spec = kr.readSpectrum(sfile)
        spec.name = src
        spt = kr.classifyTemplate(spec,plot_file='{}/classify{}_{}_{}.pdf'.format(par['REDUCTION_FOLDER'],par['MODE'],src,par['DATE']),verbose=False,fit_range=[6200,8700])
    else: print('Could not find file {}'.format(sfile))


# Blue side

In [ ]:
# BLUE SIDE PROFILE CHECK
input_file = dfolder+'../reduction/input_blue.txt'
par = kr.profileCheck(input_file,verbose=True)

In [ ]:
# REDUCE BLUE SIDE
input_file = dfolder+'../reduction/input_blue.txt'
redux = kr.reduce(input_file,reset=True,verbose=True)

In [ ]:
# CLASSIFY BLUE SIDE
input_file = dfolder+'../reduction/input_blue.txt'
kr.initializeStandards([50,75])
par = kr.readInstructions(input_file)
sources = list(par['SOURCE'].keys())
for src in sources:
    sfile = '{}/kast{}_{}_{}.fits'.format(par['REDUCTION_FOLDER'],par['MODE'],src,par['DATE'])
    if os.path.exists(sfile):
        spec = kr.readSpectrum(sfile)
        spec.name = src
        spt = kr.classifyTemplate(spec,plot_file='{}/classify{}_{}_{}.pdf'.format(par['REDUCTION_FOLDER'],par['MODE'],src,par['DATE']),verbose=False,fit_range=[4500,5300])
    else: print('Could not find file {}'.format(sfile))
